# Overview

In order to answer most of the customer usage and business insights related questions raised in the OpenShift Workload Fingerprinting project, we need to connect two disparate datasets - the insights operator archive and the pyxis database. That is, we want to use pyxis to determine the product name, architecture, vulnerabilities, vendor, etc corresponding to the container image SHA’s in the insights dataset. In a previous [issue](https://github.com/aicoe-aiops/openshift-workload-fingerprinting/issues/11#issuecomment-867776402), we figured out how to do this for a given SHA, by using curl in the terminal. In this notebook, we will try to do this programmatically, and do it for all the SHA’s available in our dataset. We will then store this merged dataset to an s3 bucket and use it for the rest of the analysis in the project going forward. 

# Pre-requisite

In order to fetch the image name (and other details) for the given 'sha' of the image_id, please complete the pre-requisite described below.

1. Follow the [link](https://source.redhat.com/groups/public/ccs-onboarding-program/ccs_onboarding_wiki/setting_up_a_kerberos_ticket_and_red_hat_idm) in order to set-up a kerberos ticket and Red Hat IdM on your machine.

2. Update the `/etc/krb5.conf` on your machine by setting `dns_canonical_hostname` to `false`, as described in the first 'red box' in this [guide](https://docs.engineering.redhat.com/display/HSSP/Pyxis+access+request)

3. Obtain the kerberos ticket by running, `$ kinit <your_kerberos_username>@IPA.REDHAT.COM`


In this notebook, we map the given "sha" of the image_id for the image layer dataset and container dataset provided from the workload data of the insight operator. 

In [165]:
#!pipenv install BeautifulSoup4
#!pipenv install pyarrow
#!pipenv install progressbar
#!pipenv install boto3
#!pipenv install tqdm

# Importing useful packages

In [1]:
import io
import boto3
import requests
import os
import json
import warnings
import pandas as pd

# from bs4 import BeautifulSoup
from requests_kerberos import HTTPKerberosAuth, OPTIONAL
from dotenv import find_dotenv, load_dotenv
from tqdm import tqdm

In [2]:
load_dotenv(find_dotenv())
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

# Data Collection

In this section, we will fetch from our s3 bucket the containers dataset and the image layers dataset that have been curated from insights operator archives. To learn more about the general content of datasets, please check out the [getting_started](https://github.com/aicoe-aiops/openshift-workload-fingerprinting/blob/master/notebooks/getting_started_notebook.ipynb) notebook. 

In [3]:
# CEPH Bucket variables
s3_endpoint_url = os.getenv("S3_ENDPOINT")
s3_access_key = os.getenv("S3_ACCESS_KEY")
s3_secret_key = os.getenv("S3_SECRET_KEY")
s3_bucket = os.getenv("S3_BUCKET")

# s3 resource to communicate with storage
s3 = boto3.resource(
    "s3",
    endpoint_url=s3_endpoint_url,
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
)

# access the parquet file as an s3 object

obj1 = s3.Object(
    "DH-PLAYPEN", "ccx/fingerprinting/image_layers/date=2021-05-12/2021-05-12.parquet"
)
obj2 = s3.Object(
    "DH-PLAYPEN", "ccx/fingerprinting/containers/date=2021-05-12/2021-05-12.parquet"
)
# download the file into the buffer
buffer1 = io.BytesIO()
obj1.download_fileobj(buffer1)
buffer2 = io.BytesIO()
obj2.download_fileobj(buffer2)

# read the buffer and create the dataframe
image_layers_df = pd.read_parquet(buffer1)
containers_df = pd.read_parquet(buffer2)

In [4]:
image_layers_df.head(3)

,cluster_id,image_id,layer_image_id,layer_image_level,first_command,first_arg,archive_path
0,00003d61-9db1-4757-9cd1-84df271daeb9,sha256:337c22cabe530213b14965f9ea69a92dbeb5104...,sha256:9ebb302e1fb002fb643091710dac46f8258781d...,0,icTsn2s_EIax,2v1NneeWoS_9,archives/compressed/00/00003d61-9db1-4757-9cd1...
1,00003d61-9db1-4757-9cd1-84df271daeb9,sha256:337c22cabe530213b14965f9ea69a92dbeb5104...,sha256:a74396a32e85c2feeedf76052ed3297859810c8...,1,icTsn2s_EIax,2v1NneeWoS_9,archives/compressed/00/00003d61-9db1-4757-9cd1...
2,00003d61-9db1-4757-9cd1-84df271daeb9,sha256:337c22cabe530213b14965f9ea69a92dbeb5104...,sha256:7db62383a7470afbacfc0fab55d5a182e3c5fa2...,2,icTsn2s_EIax,2v1NneeWoS_9,archives/compressed/00/00003d61-9db1-4757-9cd1...


In [5]:
containers_df.head(3)

,cluster_id,namespace,shape,shape_instances,image_id,first_command,first_arg,init_container,archive_path
0,00003d61-9db1-4757-9cd1-84df271daeb9,0LiT6ZNtbpYL,sha256:3ecf29979b2722bf4a82a5e7a954e8685820720...,1,sha256:f46f210d6023bec16e68340b484a8881ce46d5e...,None,47DEQpj8HBSa,False,archives/compressed/00/00003d61-9db1-4757-9cd1...
1,00003d61-9db1-4757-9cd1-84df271daeb9,0LiT6ZNtbpYL,sha256:3ecf29979b2722bf4a82a5e7a954e8685820720...,1,sha256:edb9aaacf421c6dc45b20324e8699cec02f26bf...,n9CdwzVF-cwZ,RNOaw_AuQeIY,False,archives/compressed/00/00003d61-9db1-4757-9cd1...
2,00003d61-9db1-4757-9cd1-84df271daeb9,0LiT6ZNtbpYL,sha256:542d007d13008cc1be2dbf03601b954c4452947...,1,sha256:a693c315b775c693dc49c19b7f217762676bc28...,b51B0EZ1bw3c,ua-xlwwsvdYd,False,archives/compressed/00/00003d61-9db1-4757-9cd1...


## Mapping the SHA's in `image_id` column of Image layers Dataset

First, we try to form a list of unique image_id from the image layer dataset. Using that list, we will be doing the web scraping followed by the formation of the dataframe with image_id and corresponding product name, summary, vendor, version, and other attributes.

In [10]:
# Creating the list of image_id
arr_imageid = image_layers_df.image_id.unique()
list_imageid = arr_imageid.tolist()

In [11]:
# Size of the image_id list
len(list_imageid[:])

256

In [12]:
kerberos_auth = HTTPKerberosAuth(mutual_authentication=OPTIONAL)

In [174]:
dataframe = pd.DataFrame([])

for i in tqdm(range(len(list_imageid))):
    base_url = "https://pyxis.engineering.redhat.com/v1/images?filter=image_id=="
    image_id = list_imageid[i]
    team_url = base_url + str(image_id)
    # print(team_url)
    r = requests.get(team_url, auth=kerberos_auth, verify=False)
    data = json.loads(r.content)
    if len(data["data"]) > 0:
        if len(data["data"][0]["parsed_data"]["labels"]) > 0:
            df = pd.DataFrame(data["data"][0]["parsed_data"]["labels"])
            table = pd.pivot_table(
                df, values="value", aggfunc=lambda x: x, columns="name"
            )
            table["image_id"] = list_imageid[i]
            table = table.set_index("image_id")
            dataframe = dataframe.append(table)

100%|██████████| 256/256 [04:20<00:00,  1.02s/it]


In [175]:
dataframe_image_id = dataframe
dataframe_image_id.head()

,License,architecture,build-date,com.redhat.build-host,com.redhat.component,com.redhat.license_terms,description,distribution-scope,io.k8s.description,io.k8s.display-name,io.openshift.build.commit.id,io.openshift.build.commit.url,io.openshift.build.source-location,io.openshift.expose-services,io.openshift.maintainer.component,io.openshift.maintainer.product,io.openshift.tags,maintainer,name,release,summary,url,vcs-ref,vcs-type,vendor,version,io.openshift.s2i.scripts-url,io.s2i.scripts-url,usage,io.openshift.s2i.assemble-user,authoritative-source-url,io.fabric8.s2i.version.jolokia,org.concrt.version,org.jboss.product,org.jboss.product.amq.version,org.jboss.product.openjdk.version,org.jboss.product.version,com.redhat.deployments-dir,com.redhat.dev-mode,io.jenkins.version
image_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
sha256:337c22cabe530213b14965f9ea69a92dbeb5104f79b00fd7861db7057a9e05ae,GPLv2+,x86_64,2021-04-30T00:31:42.349887,cpt-1004.osbs.prod.upshift.rdu2.redhat.com,ose-cli-artifacts-container,https://www.redhat.com/agreements,"OpenShift is a platform for developing, buildi...",public,"OpenShift is a platform for developing, buildi...",OpenShift Clients,a765590e1b87b014b9d81f9ea534460d6dff73f2,https://github.com/openshift/oc/commit/a765590...,https://github.com/openshift/oc,,oc,OpenShift Container Platform,"openshift,cli","Red Hat, Inc.",openshift/ose-cli-artifacts,202104292348.p0,Provides the latest release of Red Hat Univers...,https://access.redhat.com/containers/#/registr...,43f412b2932a5ecfe5ebedcab38cf2915cf03813,git,"Red Hat, Inc.",v4.8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sha256:3574d6c1fcc46e1ebd41b7b887b92035ea18213133c06edad539cde25b767ff9,GPLv2+,x86_64,2021-04-30T00:22:29.539443,cpt-1008.osbs.prod.upshift.rdu2.redhat.com,openshift-enterprise-cli-container,https://www.redhat.com/agreements,"OpenShift is a platform for developing, buildi...",public,"OpenShift is a platform for developing, buildi...",OpenShift Client,a765590e1b87b014b9d81f9ea534460d6dff73f2,https://github.com/openshift/oc/commit/a765590...,https://github.com/openshift/oc,,oc,OpenShift Container Platform,"openshift,cli","Red Hat, Inc.",openshift/ose-cli,202104292348.p0,Provides the latest release of Red Hat Univers...,https://access.redhat.com/containers/#/registr...,d3ae20f4e2fac18ea671cf12636d16791146a460,git,"Red Hat, Inc.",v4.8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sha256:c588b4653ef037b1300f9692b986cbeac5dfe5e66f6605ca13e0565202e8fe02,GPLv2+,x86_64,2021-05-04T21:53:04.228477,cpt-1005.osbs.prod.upshift.rdu2.redhat.com,golang-github-openshift-oauth-proxy-container,https://www.redhat.com/agreements,OpenShift OAuth Proxy.,public,OpenShift OAuth Proxy.,OpenShift OAuth Proxy,4bd4705011c19a7556551c9a1dfaaa5eacc7898c,https://github.com/openshift/oauth-proxy/commi...,https://github.com/openshift/oauth-proxy,,Release,OpenShift Container Platform,oauth,"Red Hat, Inc.",openshift/ose-oauth-proxy,202105042126.p0,Provides the latest release of Red Hat Univers...,https://access.redhat.com/containers/#/registr...,6f8aceb12953fba6382c52a2a99200bcab6c6f6b,git,"Red Hat, Inc.",v4.8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sha256:7b879d0179b96fe682a186af60836e4091f1fd3148d4cc5e9b10b3fc65bd2900,GPLv2+,x86_64,2021-04-29T23:09:36.775464,cpt-1008.osbs.prod.upshift.rdu2.redhat.com,golang-github-openshift-oauth-proxy-container,https://www.redhat.com/agreements,OpenShift OAuth Proxy.,public,OpenShift OAuth Proxy.,OpenShift OAuth Proxy,4bd4705011c19a7556551c9a1dfaaa5eacc7898c,https://github.com/openshift/oauth-proxy/commi...,https://github.com/openshift/oauth-proxy,,Release,OpenShift Container Platform,oauth,"Red Hat, Inc.",openshift/ose-oauth-proxy,202104292205.p0,Provides the latest release of Red Hat Univers...,https://access.redhat.com/containers/#/registr...,1bbf5dd3c77a259d830eb5182c2f5cf0f6431a20,git,"Red Hat, Inc.",v4.8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sha256:85a79d95924e4a76b1a6ccd9

In [176]:
dataframe_image_id.shape

(111, 40)

We were able to create a dataframe which maps 111 sha's of the image id provided in the image layer dataset out of 256 sha's in the image layer dataset.

In [6]:
# Uploading the mapping dataset in the DH-PLAYPEN bucket
parquet_buffer = io.BytesIO()
dataframe_image_id.to_parquet(parquet_buffer)
s3_obj = s3.Object(
    s3_bucket, "ccx/fingerprinting/image_layers/dataframe_image_id.parquet"
)
status = s3_obj.put(Body=parquet_buffer.getvalue())

'\n# Uploading the mapping dataset in the DH-PLAYPEN bucket\nparquet_buffer = io.BytesIO()\ndataframe_image_id.to_parquet(parquet_buffer)\ns3_obj = s3.Object(\n    s3_bucket, "ccx/fingerprinting/image_layers/dataframe_image_id.parquet"\n)\nstatus = s3_obj.put(Body=parquet_buffer.getvalue())\n'

The corresponding image_id mapped with the product name is saved in the DH-PLAYPEN bucket in the form of dataframe (_dataframe_image_id.parquet_).  

---

# Mapping SHA's from `image_layer_id` column of Image Layers Dataset

In addition to the `image_id` column, the `image_layer_id` column also contains image SHA's. These SHA's correspond to the layers that make up the image in `image_id`. In this section, we try to form a list of unique image_layer_id from the image layer dataset. Using that list, we will be doing the web scraping followed by the formation of the dataframe with image_layer_id and corresponding product name, summary, vendor, and other attributes.

In [13]:
arr_layer_imageid = image_layers_df.layer_image_id.unique()
list_layer_imageid = arr_layer_imageid.tolist()

In [14]:
# Size of the imae_layers list
len(list_layer_imageid)

752

In [15]:
dataframe = pd.DataFrame([])

for i in tqdm(range(len(list_layer_imageid))):
    base_url = "https://pyxis.engineering.redhat.com/v1/images?filter=top_layer_id=="
    image_id = list_layer_imageid[i]
    team_url = base_url + str(image_id)
    # print(team_url)
    r = requests.get(team_url, auth=kerberos_auth, verify=False)
    data = json.loads(r.content)
    if len(data["data"]) > 0:
        if len(data["data"][0]["parsed_data"]["labels"]) > 0:
            df = pd.DataFrame(data["data"][0]["parsed_data"]["labels"])
            table = pd.pivot_table(
                df, values="value", aggfunc=lambda x: x, columns="name"
            )
            table["image_id"] = list_layer_imageid[i]
            table = table.set_index("image_id")
            dataframe = dataframe.append(table)

100%|██████████| 752/752 [11:26<00:00,  1.10it/s]


In [16]:
df_image_layerid = dataframe
df_image_layerid.head()

,architecture,build-date,com.redhat.build-host,com.redhat.component,com.redhat.license_terms,description,distribution-scope,io.k8s.description,io.k8s.display-name,io.openshift.expose-services,io.openshift.tags,maintainer,name,release,summary,url,vcs-ref,vcs-type,vendor,version,io.openshift.s2i.scripts-url,io.s2i.scripts-url,com.redhat.deployments-dir,com.redhat.dev-mode,com.redhat.dev-mode.port,help,usage,io.openshift.s2i.assemble-user,authoritative-source-url,License,Architecture,Authoritative_Registry,BZComponent,Build_Host,Name,Release,Vendor,Version,com.ibm.hdm.common.branch,com.ibm.hdm.common.buildnumber,com.ibm.hdm.common.commitId,org.label-schema.vcs-ref,org.label-schema.vcs-url,io.openshift.build.commit.id,io.openshift.build.commit.url,io.openshift.build.source-location,io.openshift.maintainer.product,io.openshift.maintainer.component,io.cekit.version,io.fabric8.s2i.version.jolokia,io.fabric8.s2i.version.maven,io.openshift.s2i.destination,org.jboss.container.deployments-dir,org.jboss.product,org.jboss.product.openjdk.version,org.jboss.product.version,org.concrt.version,org.jboss.product.amq.version,com.ibm.license_terms,org.label-schema.build-date,org.label-schema.license,org.label-schema.name,org.label-schema.schema-version,org.label-schema.vendor,io.fabric8.s2i.version.karaf,io.fabric8.s2i.version.prometheus.jmx_exporter,org.jboss.deployments-dir,org.jboss.product.eap.version
image_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
sha256:fdb393d8227cbe9756537d3f215a3098ae797bd4bde422aaa10ebde84a940893,x86_64,2021-02-10T00:15:26.710361,cpt-1004.osbs.prod.upshift.rdu2.redhat.com,ubi8-container,https://www.redhat.com/en/about/red-hat-end-us...,The Universal Base Image is designed and engin...,public,The Universal Base Image is designed and engin...,Red Hat Universal Base Image 8,,base rhel8,"Red Hat, Inc.",ubi8,289,Provides the latest release of Red Hat Univers...,https://access.redhat.com/containers/#/registr...,7dd0869dabeebe7119fc04b16767c5991bcaa865,git,"Red Hat, Inc.",8.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sha256:64607cc74f9cbe0e12f167547df0cf661de5a8b1fb4ebe930a43b9f621ca457f,x86_64,2021-03-30T18:30:01.465792,cpt-1003.osbs.prod.upshift.rdu2.redhat.com,ubi8-container,https://www.redhat.com/en/about/red-hat-end-us...,The Universal Base Image is designed and engin...,public,The Universal Base Image is designed and engin...,Red Hat Universal Base Image 8,,base rhel8,"Red Hat, Inc.",ubi8,297,Provides the latest release of Red Hat Univers...,https://access.redhat.com/containers/#/registr...,7dd0869dabeebe7119fc04b16767c5991bcaa865,git,"Red Hat, Inc.",8.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sha256:7ca359fa9862549d7254cc7410b8f5e44c54412eac6c83fdd9fa8ea8710471da,x86_64,2020-12-08T19:08:54.854539,cpt-1001.osbs.prod.upshift.rdu2.redhat.com,ubi7-container,https://www.redhat.com/en/about/red-hat-end-us...,The Universal Base Image is designed and engin...,public,The Universal Base Image is designed and engin...,Red Hat Universal Base Image 7,NaN,base rhel7,NaN,ubi7,254,Provides the latest release of the Red Hat Uni...,https://access.redhat.com/containers/#/registr...,38612b4b528ab0e3177b94491bb147cfbccfdefe,git,"Red Hat, Inc.",7.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sha256:d497c3fc656f819799726c642c2c36405ea113e80eccba9aab306cb31b8632d2,x86_64,2020-12-16T00:02:32.230857,cpt-1006.osbs.prod.upshift.rdu2.redhat.com,s2i-core-container,https://www.redhat.com/en/about/red-hat-end-us...,The s2i-core image provides any images layered...,public,The s2i-co

In [18]:
df_image_layerid.shape

(143, 68)

We were able to create a dataframe which maps 143 sha's of the image layer id provided in the image layer dataset out of 752 sha's in the image layer dataset. These mapped datasets are then stored in the DH_PLAYPEN bucket.

In [19]:
# Uploading the mapping dataset in the DH-PLAYPEN bucket
parquet_buffer = io.BytesIO()
df_image_layerid.to_parquet(parquet_buffer)
s3_obj = s3.Object(
    s3_bucket, "ccx/fingerprinting/image_layers/df_image_layerid.parquet"
)
status = s3_obj.put(Body=parquet_buffer.getvalue())

---

# Mapping the SHA's in`image id` column of Containers dataset

In this section, we will map the SHA's in the containers dataset to their product name, summary, vendor, and other attributes. We will first form a list of unique image_id's from the containers dataset. Using that list, we will be doing the web scraping followed by the formation of the dataframe with image_id and corresponding attributes.

In [192]:
# Listing out the SHA's of image_id
arr_cont_imageid = containers_df.image_id.unique()
list_cont_imageid = arr_cont_imageid.tolist()

In [193]:
dataframe = pd.DataFrame([])

for i in tqdm(range(len(list_cont_imageid))):
    base_url = "https://pyxis.engineering.redhat.com/v1/images?filter=image_id=="
    image_id = list_cont_imageid[i]
    team_url = base_url + str(image_id)
    # print(team_url)
    r = requests.get(team_url, auth=kerberos_auth, verify=False)
    data = json.loads(r.content)
    if len(data["data"]) > 0:
        if len(data["data"][0]["parsed_data"]["labels"]) > 0:
            df = pd.DataFrame(data["data"][0]["parsed_data"]["labels"])
            table = pd.pivot_table(
                df, values="value", aggfunc=lambda x: x, columns="name"
            )
            table["image_id"] = list_cont_imageid[i]
            table = table.set_index("image_id")
            dataframe = dataframe.append(table)

100%|██████████| 5477/5477 [1:25:56<00:00,  1.06it/s]  


In [194]:
df_cont_image_id = dataframe
df_cont_image_id.head()

,License,architecture,build-date,com.redhat.build-host,com.redhat.component,com.redhat.license_terms,description,distribution-scope,io.k8s.description,io.k8s.display-name,io.openshift.build.commit.id,io.openshift.build.commit.url,io.openshift.build.source-location,io.openshift.expose-services,io.openshift.maintainer.component,io.openshift.maintainer.product,io.openshift.tags,maintainer,name,release,summary,url,vcs-ref,vcs-type,vendor,version,io.openshift.maintainer.subcomponent,io.openshift.release.operator,io.openshift.build.versions,com.redhat.delivery.appregistry,upstream-vcs-ref,upstream-vcs-type,upstream-version,org.kubevirt.hco.csv-generator.v1,io.openshift.s2i.scripts-url,io.s2i.scripts-url,usage,io.openshift.s2i.assemble-user,display-name,com.redhat.delivery.operator.bundle,com.redhat.openshift.versions,io.cekit.version,operators.operatorframework.io.bundle.channel.default.v1,operators.operatorframework.io.bundle.channels.v1,operators.operatorframework.io.bundle.manifests.v1,operators.operatorframework.io.bundle.mediatype.v1,operators.operatorframework.io.bundle.metadata.v1,operators.operatorframework.io.bundle.package.v1,licenses,CEPH_POINT_RELEASE,GIT_BRANCH,GIT_CLEAN,GIT_COMMIT,GIT_REPO,RELEASE,ocs.tags,com.redhat.deployments-dir,com.redhat.dev-mode,com.redhat.dev-mode.port,help,operators.operatorframework.io.index.database.v1,authoritative-source-url,license,io.fabric8.s2i.version.jolokia,io.fabric8.s2i.version.maven,io.openshift.s2i.destination,org.jboss.container.deployments-dir,org.jboss.product,org.jboss.product.eap.version,org.jboss.product.openjdk.version,org.jboss.product.sso.version,org.jboss.product.version,istio_version,openshift_build,operator_build,run,org.concrt.version,org.jboss.product.amq.version,io.openshift.build.commit.author,io.openshift.build.commit.date,io.openshift.build.commit.message,io.openshift.build.commit.ref,io.openshift.build.name,io.openshift.build.namespace,io.openshift.build.source-context-dir,jenkins.build.number,jenkins.tarball.url,io.jenkins.version,build-utility,org.label-schema.vcs-ref,org.label-schema.vcs-url,org.label-schema.description,org.label-schema.license,org.label-schema.name,org.label-schema.schema-version,org.label-schema.vendor,JAVA_VERSION,com.ibm.events.commitid,com.ibm.eventstreams.base-for-bedrock.icp-linux-amd64.commitid,com.ibm.eventstreams.base-for-bedrock.icp-linux-amd64.job,com.ibm.eventstreams.base-for-bedrock.icp-linux-amd64.license,com.ibm.eventstreams.base-for-bedrock.icp-linux-amd64.maintainer,com.ibm.eventstreams.base-for-bedrock.icp-linux-amd64.name,com.ibm.eventstreams.base-for-bedrock.icp-linux-amd64.version,com.ibm.eventstreams.openjdk-11-sdk-for-bedrock.icp-linux-amd64.commitid,com.ibm.eventstreams.openjdk-11-sdk-for-bedrock.icp-linux-amd64.job,com.ibm.eventstreams.openjdk-11-sdk-for-bedrock.icp-linux-amd64.license,com.ibm.eventstreams.openjdk-11-sdk-for-bedrock.icp-linux-amd64.maintainer,com.ibm.eventstreams.openjdk-11-sdk-for-bedrock.icp-linux-amd64.name,com.ibm.eventstreams.openjdk-11-sdk-for-bedrock.icp-linux-amd64.version,com.redhat.apb.runtime,com.microsoft.product,com.microsoft.version
image_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
sha256:f46f210d6023bec16e68340b484a8881ce46d5e6749f7c7f86f9840709d2e080,ASL 2.0,x86_64,2021-05-04T21:54:58.392948,cpt-1007.osbs.prod.upshift.rdu2.redhat.com,kube-rbac-proxy-container,https://www.redhat.com/agreements,"This is a proxy, that can perform Kubernetes R...",public,"This is a proxy, that can perform Kubernetes R...",kube-rbac-proxy,8d11a8fa9ce252cd25794c0d9280cbdc0c2affcb,https://github.com/openshift/kube-rbac-proxy/c...,https://github.com/openshift/kube-rbac-proxy,,Monitoring,OpenShift Container Platform,kubernetes,OpenShift Monitoring Team <team-monitoring@red...,openshift/ose-kube-rbac-proxy,202105042126.p0,,https://access.redhat.com/containers/#/registr...,12ef9d3cc226f6bd4a898d4b23ffa1ec5d3d27f1,git,"Red Hat, Inc.",v4.8.0,

In [197]:
df_cont_image_id.shape

(2478, 113)

Here, we successfully did the mapping for 2478 sha's of the image_id for container dataset out of 5477 sha's of image_id.

In [198]:
# Uploading the mapping dataset in the DH-PLAYPEN bucket
parquet_buffer = io.BytesIO()
df_cont_image_id.to_parquet(parquet_buffer)
s3_obj = s3.Object(s3_bucket, "ccx/fingerprinting/containers/df_cont_image_id.parquet")
status = s3_obj.put(Body=parquet_buffer.getvalue())

The corresponding mapped dataframe is saved in DH-PLAYPEN bucket.

---

# Conclusion

The notebook does takes some time to run. In the notebook, we were able to map the product name with the corresponding image_id from the image layer dataset and the container dataset. They mapped dataframe are then saved in the DH-PLAYPEN bucket. 

As next steps, in forthcoming notebooks we will integrate the dataframe generated here with the sample insight operator workload dataframe, and perfrom EDA in context of the combined information.